In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pymongo

In [2]:
# Database setup
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['food_analysis']
recipes = db['recipes']

In [3]:
choco_chip = pd.DataFrame(list(recipes.find({"title":{"$regex":"chocolate chip", "$options" :'i'}})))

In [5]:
choco_chip = pd.read_csv('choco_chip.csv')

In [ ]:
choco_chip

In [39]:
# Ingredient Reference
ir = {"flour":{
    "types":['all-purpose', "white", "brown", "spelt", "oat", "millet"
             "whole-wheat", "almond", "tapioca", "sorghum", "rice", "coconut", "garfava"
            "barley", "cake", "bread", "self-rising", "soy", "pastry"],
    "common_units":["cup", "c.", "oz", "gram", "g"]
}}

# Quantities
units = ["cup", "c." "g", "gram", 'teaspoon', "tsp", "tbsp", "oz", 'tablespoon','container','packet','bag', "stick",
         'quart','pound','can','bottle', 'pint','package','ounce','jars','heads','gallons','drops',
         'envelope','bar','box','pinch', 'dash','bunch','recipe','layer','slice','link','bulb','stalk','square','sprig',
        'fillet','piece','leg','thigh','cube','granule','strip','tray','leave','loaves','halves']

fractions = {"↉":"0", "⅒":"1/10", "⅑":"1/9", "⅛":"1/8", 
             "⅐":"1/7", "⅙":"1/6", "⅕":"1/5", "¼":"1/4", 
             "⅓":"1/3", "½":"1/2", "⅖":"2/3", "⅔":"2/3", 
             "⅜":"3/8", "⅗":"3/5", "¾":"3/4", "⅘":"4/5", 
             "⅝":"5/8", "⅚":"5/6", "⅞":"7/8"}

In [14]:
# The staples
choco_chip['ingredients'] = choco_chip.ingredients.replace(fractions)

In [36]:
import ast
ings = []
for ing_list in choco_chip.ingredients:
    ings.extend(ast.literal_eval(ing_list))

In [123]:
def parse_quant(quant):
    
    if quant is None:
        return 0
    
    fractions = {"↉":"0", "⅒":"1/10", "⅑":"1/9", "⅛":"1/8", 
                 "⅐":"1/7", "⅙":"1/6", "⅕":"1/5", "¼":"1/4", 
                 "⅓":"1/3", "½":"1/2", "⅖":"2/3", "⅔":"2/3", 
                 "⅜":"3/8", "⅗":"3/5", "¾":"3/4", "⅘":"4/5", 
                 "⅝":"5/8", "⅚":"5/6", "⅞":"7/8"}
    
    new_quant = quant.replace("⁄", "/")
    for frac in fractions:
        if frac in quant:
            new_quant.replace(frac, fractions[frac])
            
    quant_num = 0
    for num in new_quant.split(" "):
        try:
            if "/" in num:
                try:
                    quant_num += ast.literal_eval(num.split("/")[0])/ast.literal_eval(num.split("/")[1])
                except:
                    quant_num += 0
            else:
                quant_num += ast.literal_eval(num)
        except:
            quant_num += 0
    
    return quant_num

In [124]:
def parse_ingredient(ing):
    
    ing_dict = {"ingredient_string":ing,
                    "quant":None,
                    "unit":None,
                    "item":None,
                    "descriptors":None}
    split_ing = [None, None]
    
    for unit in units:
        if " "+unit+"s " in ing:
            ing_dict["unit"] = unit
            split_ing = ing.split(" "+unit+"s ")
            break
        elif " "+unit+"es " in ing:
            ing_dict["unit"] = unit
            split_ing = ing.split(" "+unit+"es ")
            break
        elif " "+unit+" " in ing:
            ing_dict["unit"] = unit
            split_ing = ing.split(" "+unit+" ")
            break
        else:
            continue
    
    # Assign
    ing_dict["quant"] = parse_quant(split_ing[0])
    ing_dict["item"] = split_ing[1]
            
    # Special
    if "egg" in ing:
        ing_dict['item'] = "egg"
        ing_dict["unit"] = "eggs"
        ing_dict["quant"] = parse_quant(ing.split(" ")[0])
    
    return ing_dict

In [125]:
all_ings = {}
for i, ing in enumerate(ings):
    all_ings[i] = parse_ingredient(ing)

In [13]:
for recipe in choco_chip.ingredients:
    for ing_ in recipe:
        ing = ing_.lower()
        split=[]
        if "flour" in ing:
            for unit in ["cup", "c.", "oz", "gram", "g"]:
                if unit+"s" in ing:
                    split = ing.split(" "+"cup"+"s ")
                else:
                    split = ing.split(" "+"cup"+" ")
                
        if len(split) > 1:
            quant = split[0]
            desc = split[1]
            
            
        

SyntaxError: can't assign to function call (<ipython-input-13-9e2c56ef5d13>, line 2)